## 1. 环境与依赖检查

如果你还没有按照 `requirements.txt` 安装依赖，可以使用下面的命令（可按需执行一次）：

```bash
pip install -r requirements.txt
```

在 Notebook 中也可以使用：

```python
# %pip install -r requirements.txt
```


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.chatanywhere.tech/v1")
LLM_MODEL = os.getenv("LLM_MODEL", "gpt-4o-mini")

print("📦 环境变量检查：")
print("  OPENAI_API_KEY:", "已配置" if OPENAI_API_KEY else "未配置 (将使用 DummyLLM)")
print("  SERPAPI_API_KEY:", "已配置" if SERPAPI_API_KEY else "未配置 (搜索工具将为占位实现)")
print("  OPENAI_BASE_URL:", OPENAI_BASE_URL)
print("  LLM_MODEL:", LLM_MODEL)

IS_DEEPSEEK = "deepseek.com" in (OPENAI_BASE_URL or "").lower()
if IS_DEEPSEEK:
    lm_lower = (LLM_MODEL or "").lower()
    if lm_lower not in ("deepseek-chat", "deepseek-reasoner"):
        new_model = "deepseek-reasoner" if ("reason" in lm_lower or "think" in lm_lower) else "deepseek-chat"
        print(f"ℹ️ 检测到 DeepSeek 接口，自动将模型名 '{LLM_MODEL}' 规范为 '{new_model}'。")
        LLM_MODEL = new_model


## 2. LLM 初始化（与脚本版本保持一致）

这里复用脚本中的三层回退策略：
1. 首选 `langchain_openai.ChatOpenAI`。
2. 失败时退到简易 HTTP 客户端 `HTTPFallbackChat`（直接调用 OpenAI 兼容接口）。
3. 再失败使用 `DummyLLM`，只输出占位文本，方便在无 Key 情况下调试流程。


In [ ]:
import json
import requests

class DummyLLM:
    """缺失 API Key 时使用的占位 LLM，实现 .invoke 接口。"""
    def __init__(self):
        self.model_name = "dummy-llm"
    def invoke(self, prompt):
        if isinstance(prompt, dict):
            return f"[DummyLLM response for keys: {list(prompt.keys())}]"
        return "[DummyLLM generic response]"
    def __or__(self, other):  # 兼容链式调用
        return other


class HTTPFallbackChat:
    """直接调用 OpenAI 兼容接口的轻量客户端，实现 .invoke(dict|str)。"""
    def __init__(self, base_url: str, api_key: str, model: str, timeout: float = 30.0):
        self.base_url = base_url.rstrip('/')
        self.api_key = api_key
        self.model = model
        self.timeout = timeout
        if self.base_url.endswith('/v1'):
            self.endpoint = f"{self.base_url}/chat/completions"
        else:
            self.endpoint = f"{self.base_url}/v1/chat/completions"

    def invoke(self, prompt):
        if isinstance(prompt, dict):
            user_content = '\n'.join(f"{k}: {v}" for k, v in prompt.items())
        else:
            user_content = str(prompt)
        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": "You are an academic writing optimization assistant."},
                {"role": "user", "content": user_content}
            ],
            "temperature": 0
        }
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        try:
            resp = requests.post(self.endpoint, headers=headers, json=payload, timeout=self.timeout)
            if resp.status_code != 200:
                return f"[HTTPFallbackChat Error {resp.status_code}: {resp.text[:200]}]"
            data = resp.json()
            return data.get("choices", [{}])[0].get("message", {}).get("content", "[No content]")
        except Exception as e:
            return f"[HTTPFallbackChat Exception: {e}]"

    def __or__(self, other):
        return other


def init_llm():
    if not OPENAI_API_KEY:
        print("⚠️ OPENAI_API_KEY 未配置，将使用 DummyLLM（仅用于流程调试）。")
        return DummyLLM()
    try:
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(
            model=LLM_MODEL,
            temperature=0,
            api_key=(lambda: OPENAI_API_KEY),
            base_url=OPENAI_BASE_URL,
        )
        print(f"✅ 主 LLM 已通过 ChatOpenAI 初始化: {LLM_MODEL} @ {OPENAI_BASE_URL}")
        return llm
    except Exception as e:
        print(f"❌ ChatOpenAI 初始化失败: {e}，尝试 HTTPFallbackChat...")
        try:
            fallback_llm = HTTPFallbackChat(OPENAI_BASE_URL, OPENAI_API_KEY, LLM_MODEL)
            probe = fallback_llm.invoke("probe")
            if probe.startswith("[HTTPFallbackChat Error") or probe.startswith("[HTTPFallbackChat Exception"):
                print("⚠️ HTTPFallbackChat 探测失败，回退到 DummyLLM。")
                return DummyLLM()
            print(f"✅ HTTPFallbackChat 已就绪: {LLM_MODEL} @ {OPENAI_BASE_URL}")
            return fallback_llm
        except Exception as e2:
            print(f"❌ HTTPFallbackChat 初始化失败: {e2}，回退到 DummyLLM。")
            return DummyLLM()


llm = init_llm()


## 3. 工具层：搜索 / Python REPL / 文件读写

这里与脚本版本保持一致：
- 如果没有 SerpAPI Key，则搜索工具返回占位提示，但接口仍然可用。
- 提供 Python REPL 工具执行简单代码片段。
- 提供最小的文本文件读写工具，方便在 Notebook 中快速试验。


In [ ]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.utilities import SerpAPIWrapper

TOOLS = []

if SERPAPI_API_KEY:
    search_wrapper = SerpAPIWrapper(search_engine="google", serpapi_api_key=SERPAPI_API_KEY)
    search_tool = Tool(
        name="网络搜索",
        func=search_wrapper.run,
        description="实时信息查询：输入搜索关键词"
    )
    TOOLS.append(search_tool)
else:
    def _search_stub(q: str) -> str:
        return f"[SerpAPI 未配置，无法执行搜索: {q}]"
    search_tool = Tool(name="网络搜索", func=_search_stub, description="SerpAPI 未配置，占位搜索工具")
    TOOLS.append(search_tool)

python_repl = PythonREPL()
python_repl_tool = Tool(
    name="Python REPL",
    func=python_repl.run,
    description="执行格式正确的 Python 代码"
)

read_file_tool = Tool(
    name="读取文件",
    func=lambda fn: open(fn, "r", encoding="utf-8").read(),
    description="读取指定文本文件的全部内容，输入为文件路径"
)

write_file_tool = Tool(
    name="写入文件",
    func=lambda arg: (
        (lambda filename, content: (open(filename, "w", encoding="utf-8").write(content), "写入完成")[1])
    )(*arg.split(",", 1)),
    description="写入文件内容。输入格式: 文件名,内容（英文逗号分隔）"
)

TOOLS.extend([python_repl_tool, read_file_tool, write_file_tool])

print(f"🔧 已加载工具数量: {len(TOOLS)}")


## 4. 向量记忆层：FAISS 或简易内存检索

与脚本版本保持一致：
- 优先使用 FAISS + OpenAIEmbeddings（或 DummyEmbeddings）。
- 如果 FAISS/依赖缺失，则退回到一个基于 token 重叠度的简易 `SimpleVectorStore`。


In [ ]:
from typing import List, Dict, Optional, Tuple
import re
from datetime import datetime

EMBED_DIM = 1536
USE_FAISS = True
try:
    from langchain_community.vectorstores import FAISS
    from langchain_community.docstore.in_memory import InMemoryDocstore
    import faiss
    from langchain_core.documents import Document
    from langchain_core.embeddings import Embeddings as LCEmbeddings
except Exception as e:
    print(f"⚠️ 向量存储依赖缺失: {e}，使用简易内存检索替代 FAISS。")
    USE_FAISS = False
    FAISS = None  # type: ignore
    InMemoryDocstore = None  # type: ignore
    faiss = None  # type: ignore
    try:
        from langchain_core.documents import Document
    except Exception:
        class Document:  # 最小占位实现
            def __init__(self, page_content: str, metadata: Optional[Dict] = None):
                self.page_content = page_content
                self.metadata = metadata or {}

    class LCEmbeddings:  # type: ignore
        _dim = EMBED_DIM
        def embed_query(self, x: str):
            return [0.0] * self._dim
        def embed_documents(self, xs: List[str]):
            return [[0.0] * self._dim for _ in xs]


EMBED_MODEL_NAME = os.getenv("EMBED_MODEL_NAME", "text-embedding-3-small")

class DummyEmbeddings:
    def embed_query(self, t: str):
        return [0.0] * EMBED_DIM
    def embed_documents(self, docs: List[str]):
        return [[0.0] * EMBED_DIM for _ in docs]
    def __call__(self, t: str):
        return self.embed_query(t)


if OPENAI_API_KEY:
    try:
        if IS_DEEPSEEK:
            print("ℹ️ 检测到 DeepSeek base_url，跳过 OpenAIEmbeddings，使用占位 embedding。")
            embeddings_model = DummyEmbeddings()
        else:
            from langchain_openai import OpenAIEmbeddings
            embeddings_model = OpenAIEmbeddings(
                model=EMBED_MODEL_NAME,
                api_key=(lambda: OPENAI_API_KEY),
                base_url=OPENAI_BASE_URL
            )
            print(f"✅ Embeddings 模型已初始化: {EMBED_MODEL_NAME}")
    except Exception as e:
        print(f"❌ Embeddings 初始化失败: {e}，使用 DummyEmbeddings。")
        embeddings_model = DummyEmbeddings()
else:
    embeddings_model = DummyEmbeddings()
    print("⚠️ OPENAI_API_KEY 缺失，Embeddings 使用占位向量。")

_DEF_WORD_RE = re.compile(r"[\u4e00-\u9fff]|[A-Za-z0-9_]+")

def _simple_tokenize(text: str) -> List[str]:
    return _DEF_WORD_RE.findall(text or "")


if USE_FAISS:
    class EmbeddingAdapter(LCEmbeddings):
        def __init__(self, base):
            self.base = base
        def embed_query(self, x: str) -> List[float]:
            try:
                return self.base.embed_query(x)
            except Exception:
                return [0.0] * EMBED_DIM
        def embed_documents(self, xs: List[str]) -> List[List[float]]:
            try:
                return self.base.embed_documents(xs)
            except Exception:
                return [[0.0] * EMBED_DIM for _ in xs]

    adapter = EmbeddingAdapter(embeddings_model)
    index = faiss.IndexFlatL2(EMBED_DIM)
    vectorstore = FAISS(adapter, index, InMemoryDocstore({}), {})
    print("🧠 向量数据库(FAISS)已初始化。")
else:
    class SimpleVectorStore:
        def __init__(self):
            self.docs: List[Document] = []
        def add_documents(self, docs: List[Document]):
            self.docs.extend(docs)
        def similarity_search(self, query: str, k: int = 3) -> List[Document]:
            q_tokens = set(_simple_tokenize(query))
            def score(doc: Document):
                d_tokens = set(_simple_tokenize(doc.page_content))
                if not q_tokens or not d_tokens:
                    return 0.0
                return len(q_tokens & d_tokens) / len(q_tokens | d_tokens)
            ranked = sorted(self.docs, key=score, reverse=True)
            return ranked[:k]

    vectorstore = SimpleVectorStore()
    print("🧠 向量数据库简化版已初始化（无 FAISS）。")


class MemoryManager:
    def __init__(self, vs, namespace: str = "global"):
        self.vs = vs
        self.namespace = namespace
        self._counter = 0
    def add_memory(self, text: str, metadata: Optional[Dict] = None):
        try:
            meta = metadata or {}
            meta.update({"namespace": self.namespace, "ts": datetime.now().isoformat()})
            doc = Document(page_content=text, metadata=meta)
            self.vs.add_documents([doc])
            self._counter += 1
        except Exception as e:
            print(f"⚠️ 写入记忆失败: {e}")
    def recall(self, query: str, k: int = 3) -> List[str]:
        try:
            res = self.vs.similarity_search(query, k=k)
            return [d.page_content for d in res]
        except Exception as e:
            print(f"⚠️ 读取记忆失败: {e}")
            return []


## 5. 双 Agent 学术表达优化系统（与脚本版本同步）

这里直接复用 `multi_agent_nlp_project.py` 中的实现，只保留与“协作优化”强相关的部分，方便在 Notebook 中交互式调用。


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import difflib
import time
from html import escape as _html_escape  # 新增：HTML 报告所需
try:
    from metrics import AcademicMetrics
except Exception:
    AcademicMetrics = None

# 修改与扩展 DualAgentAcademicSystem：加入双 LLM、代码块执行、数据合成/评估/蒸馏、HTML 报告、长文本优化等
class DualAgentAcademicSystem:
    def __init__(self, llm, tools, vectorstore, enable_tools: bool = True, enable_memory: bool = True,
                 agent_a_llm=None, agent_b_llm=None):
        """双 Agent 学术系统（Notebook 同步版）

        参数:
          llm: 默认 LLM，当未提供 agent_a_llm / agent_b_llm 时使用。
          tools: LangChain Tool 列表。
          vectorstore: 向量存储或简易 SimpleVectorStore。
          enable_tools: 是否启用外部工具调用。
          enable_memory: 是否启用长程记忆。
          agent_a_llm: （可选）Agent A 专用 LLM（优化者，如本地学生模型）。
          agent_b_llm: （可选）Agent B 专用 LLM（评审者，如远程教师模型）。
        """
        self.llm = llm
        self.agent_a_llm = agent_a_llm or llm
        self.agent_b_llm = agent_b_llm or llm
        self.tools_enabled = enable_tools
        self.memory_enabled = enable_memory
        self.tools = {t.name: t for t in tools}
        self.vectorstore = vectorstore
        self.memory = MemoryManager(vectorstore) if enable_memory else None
        self.collaboration_log: List[Dict] = []
        self._setup_agents()

    def _setup_agents(self):
        self.agent_a_template = PromptTemplate.from_template(
            """
你是Agent A - 学术表达优化专家。
轮次: 第{round_num}轮
用户需求: {user_requirements}
上一轮评分(若有): {last_scores}
长程记忆检索片段:
{memory_snippets}
工具观察:
{tool_observations}
待优化文本:
{text_to_optimize}
{previous_feedback}

请输出：
**优化版本：**
[优化后的完整文本]

**修改说明：**
[说明本轮修改要点，尤其针对评审提出的高优先级问题]
"""
        )
        self.agent_b_template = PromptTemplate.from_template(
            """
你是Agent B - 学术评审与对抗质询专家。
轮次: 第{round_num}轮
用户需求: {user_requirements}
优化文本:
{optimized_text}

请评审并输出(严格包含以下板块与数值)：
**本轮改进评价：**
[总体评价]

**评分(请使用JSON格式)**
{"quality": <1-10>, "rigor": <1-10>, "logic": <1-10>, "novelty": <1-10>, "priority_issues": <描述>}

**剩余主要问题：**
[...]

**下轮重点建议：**
1. [...]
2. [...]

**改进优先级：**
[高/中/低 分层列出]
"""
        )
        self.agent_a_chain = self.agent_a_template | self.agent_a_llm | StrOutputParser()
        self.agent_b_chain = self.agent_b_template | self.agent_b_llm | StrOutputParser()

    @staticmethod
    def _extract_section(text: str, start_token: str, end_token: str) -> str:
        lines = text.split('\n')
        collecting = False
        buf = []
        for l in lines:
            if start_token in l:
                collecting = True
                continue
            if collecting and end_token in l:
                break
            if collecting:
                buf.append(l)
        return '\n'.join(buf).strip()

    def _compute_diff(self, prev: str, current: str) -> str:
        if prev is None:
            return '(首轮无diff)'
        import difflib as _df
        diff_lines = _df.unified_diff(prev.splitlines(), current.splitlines(), lineterm='')
        collected = []
        for i, line in enumerate(diff_lines):
            if i > 400:
                collected.append('... <diff truncated>')
                break
            collected.append(line)
        return '\n'.join(collected) if collected else '(无变化)'

    def _parse_scores(self, feedback: str) -> Dict[str, float]:
        import json as _json, re as _re
        m = _re.search(r'\{\s*"quality".*?}', feedback, flags=_re.S)
        if not m:
            return {}
        try:
            data = _json.loads(m.group(0))
            for k in ["quality", "rigor", "logic", "novelty"]:
                if k in data:
                    data[k] = float(data[k])
            return data
        except Exception:
            return {}

    def _plan_and_act(self, text: str, requirements: List[str]) -> str:
        if not self.tools_enabled:
            return '(工具已禁用)'
        observations = []
        joined_req = ' '.join(requirements).lower()
        # 搜索触发逻辑
        if any(kw in joined_req for kw in ["search", "检索", "事实", "最新", "引用"]):
            import re as _re2
            m = _re2.findall(r'"([^\"]+)"', text)
            query = m[-1] if m else text.split('。')[-1].strip() or text
            try:
                obs = self.tools["网络搜索"].run(query)
            except Exception as e:
                obs = f"[搜索异常: {e}]"
            observations.append(f"搜索[{query}] -> {str(obs)[:300]}")
        # Python 代码块执行 (只执行首个，防止过度调用)
        import re as _re3
        code_blocks = _re3.findall(r'```python\n([\s\S]*?)```', text, flags=_re3.IGNORECASE)
        if code_blocks:
            code = code_blocks[0]
            try:
                out = self.tools.get("Python REPL").run(code)
            except Exception as e:
                out = f"[代码执行异常: {e}]"
            observations.append(f"执行Python -> 输出: {str(out)[:200]}")
        return '\n'.join(observations) if observations else '(无)'

    # ===== 基础中文分词 & 可读性/连贯性代理指标 =====
    @staticmethod
    def _tokenize_zh(text: str) -> List[str]:
        import re as _re4
        return _re4.findall(r"[\u4e00-\u9fff]|[A-Za-z0-9_]+", text)

    def _readability_proxy(self, text: str) -> float:
        import re as _re5
        sentences = [s for s in _re5.split(r'[。.!?]\s*', text) if s.strip()]
        if not sentences:
            return 0.0
        avg_len = sum(len(s) for s in sentences) / len(sentences)
        return round(1 / (1 + avg_len / 50), 4)

    def _coherence_proxy(self, text: str) -> float:
        import re as _re6
        sentences = [s for s in _re6.split(r'[。.!?]\s*', text) if s.strip()]
        if len(sentences) < 2:
            return 0.0
        def toks(s):
            return set(self._tokenize_zh(s))
        overlaps = []
        for a, b in zip(sentences[:-1], sentences[1:]):
            ta, tb = toks(a), toks(b)
            if ta and tb:
                overlaps.append(len(ta & tb) / len(ta | tb))
        return round(sum(overlaps) / len(overlaps), 4) if overlaps else 0.0

    # ===== 主协作流程 =====
    def collaborate(self, user_text: str, user_requirements: List[str], language: str = "中文", rounds: int = 3) -> Tuple[str, List[Dict]]:
        self.collaboration_log = [{"round": 0, "user_input": user_text, "requirements": user_requirements, "timestamp": datetime.now().isoformat()}]
        current_text = user_text
        previous_feedback = ""
        last_scores: Dict[str, float] = {}
        if self.memory_enabled:
            self.memory.add_memory(user_text, {"type": "user_input"})
        for r in range(1, rounds + 1):
            mem_snippets = []
            if self.memory_enabled:
                mem_snippets = self.memory.recall(current_text, k=3)
            tool_obs = self._plan_and_act(current_text, user_requirements)
            a_input = {
                "round_num": r,
                "text_to_optimize": current_text,
                "user_requirements": ', '.join(user_requirements),
                "previous_feedback": previous_feedback,
                "memory_snippets": '\n'.join(mem_snippets) if mem_snippets else '(无)',
                "tool_observations": tool_obs,
                "last_scores": last_scores if last_scores else '(无)'
            }
            a_resp = self.agent_a_chain.invoke(a_input)
            optimized_text = self._extract_section(a_resp, "**优化版本：**", "**修改说明：**") or current_text
            b_input = {
                "round_num": r,
                "optimized_text": optimized_text,
                "user_requirements": ', '.join(user_requirements)
            }
            b_resp = self.agent_b_chain.invoke(b_input)
            last_scores = self._parse_scores(b_resp)
            diff_str = self._compute_diff(current_text, optimized_text)
            if self.memory_enabled:
                self.memory.add_memory(optimized_text, {"type": "optimized_text", "round": r})
                self.memory.add_memory(b_resp, {"type": "feedback", "round": r})
            self.collaboration_log.append({
                "round": r,
                "agent_a_response": a_resp,
                "optimized_text": optimized_text,
                "agent_b_feedback": b_resp,
                "scores": last_scores,
                "tool_observations": tool_obs,
                "diff": diff_str,
                "timestamp": datetime.now().isoformat()
            })
            previous_feedback = b_resp
            current_text = optimized_text
            print(f"✅ Round {r} 完成 | 评分: {last_scores if last_scores else '{}'}")
            time.sleep(0.15)
        return current_text, self.collaboration_log

    # ===== 数据合成 (用于蒸馏教师信号) =====
    def synthesize_dataset(self, seeds: List[str], requirements: List[str], rounds: int = 3,
                           out_path: Optional[str] = None) -> str:
        from pathlib import Path as _P
        data_dir = _P("data")
        data_dir.mkdir(parents=True, exist_ok=True)
        if out_path is None:
            out_path = str(data_dir / f"synth_academic_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jsonl")
        count = 0
        with open(out_path, 'w', encoding='utf-8') as f:
            for i, seed in enumerate(seeds):
                final_text, log = self.collaborate(seed, requirements, rounds=rounds)
                record = {
                    "id": f"case_{i}",
                    "input": seed,
                    "requirements": requirements,
                    "final": final_text,
                    "log": log,
                    "created_at": datetime.now().isoformat(),
                    "teacher_signal": log[-1].get("optimized_text", final_text),
                    "scores": log[-1].get("scores", {})
                }
                import json as _json2
                f.write(_json2.dumps(record, ensure_ascii=False) + '\n')
                count += 1
        print(f"📦 合成数据已写入: {out_path} (共 {count} 条)")
        return out_path

    # ===== 蒸馏对准备 =====
    def prepare_distillation_pairs(self, jsonl_path: str, out_path: str) -> str:
        import json as _json3
        pairs = []
        with open(jsonl_path, 'r', encoding='utf-8') as f:
            for ln in f:
                if not ln.strip():
                    continue
                obj = _json3.loads(ln)
                instr = f"优化以下学术段落, 满足需求: {', '.join(obj.get('requirements', []))}\n原文: {obj.get('input', '')}"
                target = obj.get('teacher_signal', obj.get('final', ''))
                scores = obj.get('scores', {})
                pairs.append({"instruction": instr, "output": target, "scores": scores})
        with open(out_path, 'w', encoding='utf-8') as w:
            for p in pairs:
                w.write(_json3.dumps(p, ensure_ascii=False) + '\n')
        print(f"🧪 蒸馏数据已生成: {out_path} 共 {len(pairs)} 条")
        return out_path

    # ===== 评估逻辑 (含 AcademicMetrics 集成) =====
    def evaluate(self, cases: List[Tuple[str, List[str]]], rounds: int = 2) -> Dict:
        import json as _json4
        from collections import Counter
        results = []
        for idx, (text, reqs) in enumerate(cases):
            final_text, log = self.collaborate(text, reqs, rounds=rounds)
            w0 = self._tokenize_zh(text)
            w1 = self._tokenize_zh(final_text)
            len_gain = (len(w1) - len(w0)) / max(1, len(w0))
            ttr0 = len(set(w0)) / max(1, len(w0))
            ttr1 = len(set(w1)) / max(1, len(w1))
            c0 = Counter(w0)
            c1 = Counter(w1)
            rep0 = sum(x for _, x in c0.most_common(5)) / max(1, len(w0))
            rep1 = sum(x for _, x in c1.most_common(5)) / max(1, len(w1))
            readability_gain = self._readability_proxy(final_text) - self._readability_proxy(text)
            coherence_gain = self._coherence_proxy(final_text) - self._coherence_proxy(text)
            import statistics as _stats
            def _sent_lens(t: str):
                import re as _re7
                sents = [s for s in _re7.split(r'[。.!?]\s*', t) if s.strip()]
                return [len(s) for s in sents] if sents else []
            var0 = _stats.pvariance(_sent_lens(text)) if _sent_lens(text) else 0.0
            var1 = _stats.pvariance(_sent_lens(final_text)) if _sent_lens(final_text) else 0.0
            var_delta = round(var0 - var1, 3)
            def _bigram_rep(toks: List[str]):
                bigrams = [tuple(toks[i:i+2]) for i in range(len(toks)-1)]
                bc = Counter(bigrams)
                total = len(bigrams) or 1
                top = sum(v for _, v in bc.most_common(5))
                return top / total
            bigram_delta = round(_bigram_rep(w0) - _bigram_rep(w1), 3)
            last_scores = log[-1].get("scores", {}) if log else {}
            advanced_metrics = {}
            if AcademicMetrics:
                try:
                    original_eval = AcademicMetrics.overall_quality_score(text)
                    optimized_eval = AcademicMetrics.overall_quality_score(final_text)
                    orig_scores = (original_eval or {}).get('scores', {})
                    opt_scores = (optimized_eval or {}).get('scores', {})
                    advanced_metrics = {
                        'original_overall_score': float((original_eval or {}).get('overall_score', 0.0)),
                        'optimized_overall_score': float((optimized_eval or {}).get('overall_score', 0.0)),
                        'academic_formality_improvement': round(float(opt_scores.get('academic_formality', 0.0)) - float(orig_scores.get('academic_formality', 0.0)), 4),
                        'citation_completeness_improvement': round(float(opt_scores.get('citation_completeness', 0.0)) - float(orig_scores.get('citation_completeness', 0.0)), 4),
                        'novelty_improvement': round(float(opt_scores.get('novelty', 0.0)) - float(orig_scores.get('novelty', 0.0)), 4),
                        'language_fluency_improvement': round(float(opt_scores.get('language_fluency', 0.0)) - float(orig_scores.get('language_fluency', 0.0)), 4),
                        'sentence_balance_improvement': round(float(opt_scores.get('sentence_balance', 0.0)) - float(orig_scores.get('sentence_balance', 0.0)), 4),
                        'argumentation_improvement': round(float(opt_scores.get('argumentation', 0.0)) - float(orig_scores.get('argumentation', 0.0)), 4),
                        'expression_diversity_improvement': round(float(opt_scores.get('expression_diversity', 0.0)) - float(orig_scores.get('expression_diversity', 0.0)), 4),
                        'structure_completeness_improvement': round(float(opt_scores.get('structure_completeness', 0.0)) - float(orig_scores.get('structure_completeness', 0.0)), 4),
                        'tense_consistency_improvement': round(float(opt_scores.get('tense_consistency', 0.0)) - float(orig_scores.get('tense_consistency', 0.0)), 4),
                    }
                except Exception as e:
                    print(f"⚠️ 计算高级指标失败: {e}")
            results.append({
                "id": idx,
                "len_gain": round(len_gain, 3),
                "ttr_gain": round(ttr1 - ttr0, 3),
                "repetition_delta": round(rep0 - rep1, 3),
                "readability_gain": round(readability_gain, 3),
                "coherence_gain": round(coherence_gain, 3),
                "sent_var_delta": var_delta,
                "bigram_rep_delta": round(bigram_delta, 3),
                "orig_len": len(w0),
                "final_len": len(w1),
                "scores": last_scores,
                "advanced_metrics": advanced_metrics
            })
        if results:
            avg = {
                "len_gain_avg": round(sum(r["len_gain"] for r in results) / len(results), 3),
                "ttr_gain_avg": round(sum(r["ttr_gain"] for r in results) / len(results), 3),
                "repetition_delta_avg": round(sum(r["repetition_delta"] for r in results) / len(results), 3),
                "readability_gain_avg": round(sum(r["readability_gain"] for r in results) / len(results), 3),
                "coherence_gain_avg": round(sum(r["coherence_gain"] for r in results) / len(results), 3),
                "sent_var_delta_avg": round(sum(r["sent_var_delta"] for r in results) / len(results), 3),
                "bigram_rep_delta_avg": round(sum(r["bigram_rep_delta"] for r in results) / len(results), 3),
                "quality_avg": round(sum(r.get("scores", {}).get("quality", 0) for r in results) / len(results), 3),
                "rigor_avg": round(sum(r.get("scores", {}).get("rigor", 0) for r in results) / len(results), 3),
                "logic_avg": round(sum(r.get("scores", {}).get("logic", 0) for r in results) / len(results), 3),
                "novelty_avg": round(sum(r.get("scores", {}).get("novelty", 0) for r in results) / len(results), 3),
                "n": len(results)
            }
            if results[0].get("advanced_metrics"):
                adv_avg = {}
                for metric in results[0]["advanced_metrics"].keys():
                    adv_avg[f"{metric}_avg"] = round(sum(r.get("advanced_metrics", {}).get(metric, 0) for r in results) / len(results), 4)
                avg.update(adv_avg)
        else:
            avg = {"len_gain_avg": 0, "ttr_gain_avg": 0, "repetition_delta_avg": 0, "readability_gain_avg": 0, "coherence_gain_avg": 0, "sent_var_delta_avg": 0, "bigram_rep_delta_avg": 0, "quality_avg": 0, "rigor_avg": 0, "logic_avg": 0, "novelty_avg": 0, "n": 0}
        report = {"summary": avg, "cases": results}
        print("📊 评估汇总:", json.dumps(report["summary"], ensure_ascii=False))
        return report

    # ===== 长文本分段处理 =====
    def _split_long_text(self, text: str, chunk_size: int, overlap: int) -> List[str]:
        text = text.strip()
        if chunk_size <= 0:
            return [text]
        import re as _re8
        sentences = _re8.split(r'([。.!?]\s*)', text)
        buf = ''
        chunks = []
        for i in range(0, len(sentences), 2):
            seg = sentences[i]
            delim = sentences[i + 1] if i + 1 < len(sentences) else ''
            piece = seg + delim
            if len(buf) + len(piece) <= chunk_size:
                buf += piece
            else:
                if buf:
                    chunks.append(buf)
                buf = piece
        if buf:
            chunks.append(buf)
        if overlap > 0 and len(chunks) > 1:
            with_overlap = []
            prev_tail = ''
            for idx, c in enumerate(chunks):
                if idx == 0:
                    with_overlap.append(c)
                else:
                    tail = prev_tail[-overlap:] if overlap < len(prev_tail) else prev_tail
                    with_overlap.append((tail + c).strip())
                prev_tail = c
            chunks = with_overlap
        return chunks

    def optimize_text_file(self, file_path: str, requirements: List[str], rounds: int,
                           chunk_size: int, overlap: int, max_chunks: int = 0) -> Tuple[str, Dict]:
        from pathlib import Path as _P2
        p = _P2(file_path)
        if not p.exists():
            raise FileNotFoundError(f'文本文件不存在: {file_path}')
        raw = p.read_text(encoding='utf-8')
        chunks = self._split_long_text(raw, chunk_size, overlap)
        if max_chunks > 0:
            chunks = chunks[:max_chunks]
        optimized_segments = []
        segment_logs = []
        for idx, chunk in enumerate(chunks):
            print(f'🧩 处理分段 {idx+1}/{len(chunks)} (长度={len(chunk)})')
            final_seg, log = self.collaborate(chunk, requirements, rounds=rounds)
            optimized_segments.append(final_seg)
            segment_logs.append({
                'segment_index': idx,
                'original_length': len(chunk),
                'optimized_length': len(final_seg),
                'final_segment_text': final_seg,
                'round_logs': log
            })
        combined_final = '\n\n'.join(optimized_segments)
        aggregated = {
            'file': file_path,
            'chunks': len(chunks),
            'chunk_size': chunk_size,
            'overlap': overlap,
            'requirements': requirements,
            'final_text': combined_final,
            'segments': segment_logs,
        }
        return combined_final, aggregated

    # ===== HTML 报告生成 =====
    def generate_html_report(self, title: str, final_text: str, log: List[Dict], summary: Optional[Dict] = None) -> str:
        def style_block():
            return '''<style>body{font-family:Segoe UI,Arial,sans-serif;max-width:1080px;margin:32px auto;line-height:1.5}pre{background:#fafafa;border:1px solid #eee;padding:8px;white-space:pre-wrap}table{border-collapse:collapse;width:100%;margin:10px 0}td,th{border:1px solid #ccc;padding:6px 10px;font-size:13px}th{background:#f0f0f0} .score-badge{display:inline-block;padding:3px 8px;border-radius:4px;background:#004d7a;color:#fff;font-size:12px;margin:0 4px 4px 0} .diff-add{background:#e6ffe6} .diff-del{background:#ffecec;color:#900} .round-box{border:1px solid #ddd;padding:12px;margin:14px 0;border-radius:6px;background:#fafafa} .metric-value{font-weight:bold;color:#2c3e50}</style>'''
        def render_scores(scores: Dict[str, float]) -> str:
            if not scores:
                return '<span style="color:#777">无评分</span>'
            return ' '.join(f'<span class="score-badge">{k}:{v:.1f}</span>' for k,v in scores.items() if isinstance(v,(int,float)))
        def color_diff(diff_text: str) -> str:
            lines = []
            for ln in diff_text.splitlines():
                if ln.startswith('+') and not ln.startswith('+++'):
                    lines.append(f'<div class="diff-add">{_html_escape(ln)}</div>')
                elif ln.startswith('-') and not ln.startswith('---'):
                    lines.append(f'<div class="diff-del">{_html_escape(ln)}</div>')
                else:
                    lines.append(f'<div>{_html_escape(ln)}</div>')
            return '\n'.join(lines)
        parts = [f'<html><head><meta charset="utf-8"><title>{_html_escape(title)}</title>{style_block()}</head><body>']
        parts.append(f'<h1>📈 {_html_escape(title)}</h1>')
        parts.append(f'<div style="color:#555;font-size:12px">生成时间: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</div>')
        if summary:
            parts.append('<h2>📊 指标汇总</h2><table><tr><th>长度变化</th><th>词汇多样性提升</th><th>重复度下降</th><th>可读性提升</th><th>连贯性提升</th></tr>')
            parts.append('<tr>' + ''.join(f'<td>{summary.get(k,0):.4f}</td>' for k in ["len_gain_avg","ttr_gain_avg","repetition_delta_avg","readability_gain_avg","coherence_gain_avg"]) + '</tr></table>')
        parts.append('<h2>📝 最终优化文本</h2><pre>' + _html_escape(final_text) + '</pre>')
        parts.append('<h2>📂 轮次详细日志</h2>')
        for entry in log[1:]:
            parts.append('<div class="round-box">')
            parts.append(f'<h3>Round {entry.get("round")}</h3>')
            parts.append(f'<div style="color:#666;font-size:12px">时间: {entry.get("timestamp")}</div>')
            parts.append('<h4>✏️ 优化文本</h4><pre>' + _html_escape(entry.get('optimized_text','')) + '</pre>')
            parts.append('<h4>🗒 Agent B 反馈</h4><pre>' + _html_escape(entry.get('agent_b_feedback','')) + '</pre>')
            parts.append('<h4>⭐ 评分</h4>' + render_scores(entry.get('scores',{})))
            parts.append('<details><summary>🔍 文本差异 (Diff)</summary>' + color_diff(entry.get('diff','')) + '</details>')
            if entry.get('tool_observations') and entry.get('tool_observations') not in ('(无)','(工具已禁用)'):
                parts.append('<details><summary>🛠 工具调用观察</summary><pre>' + _html_escape(entry.get('tool_observations','')) + '</pre></details>')
            parts.append('</div>')
        parts.append('</body></html>')
        return '\n'.join(parts)

# 构建可选混合系统（学生模型 + 教师模型）
try:
    from hf_student_llm import HFChatLLM as _HFStudentLLM
except Exception:
    _HFStudentLLM = None

def build_hybrid_dual_agent_system(base_llm) -> DualAgentAcademicSystem:
    if _HFStudentLLM is None:
        print("⚠️ HF 学生模型不可用（缺少 transformers/peft 或导入失败），使用单一 LLM 模式。")
        return DualAgentAcademicSystem(base_llm, TOOLS, vectorstore)
    import os as _os
    from pathlib import Path as _P3
    base_model = _os.getenv("STUDENT_BASE_MODEL", "Qwen/Qwen1.5-1.8B-Chat")
    lora_dir = _os.getenv("STUDENT_LORA_DIR")
    max_new = int(_os.getenv("STUDENT_MAX_NEW_TOKENS", "512"))
    if not lora_dir or not _P3(lora_dir).exists():
        print(f"⚠️ STUDENT_LORA_DIR 未设置或路径不存在 ({lora_dir})，仅加载基础模型。")
        lora_dir = ""
    try:
        if lora_dir:
            student_llm = _HFStudentLLM(base_model=base_model, lora_dir=lora_dir, max_new_tokens=max_new)
        else:
            student_llm = _HFStudentLLM(base_model=base_model, lora_dir=base_model, max_new_tokens=max_new)
        print(f"🤖 Agent A 使用本地学生模型: {base_model} (LoRA: {bool(lora_dir)}) | Agent B 使用远程教师模型")
        return DualAgentAcademicSystem(base_llm, TOOLS, vectorstore, agent_a_llm=student_llm, agent_b_llm=base_llm)
    except Exception as e:
        print(f"⚠️ 初始化学生模型失败: {e}，退回单一 LLM 模式。")
        return DualAgentAcademicSystem(base_llm, TOOLS, vectorstore)

# 重新初始化系统为混合（若学生模型可用）
dual_agent_system = build_hybrid_dual_agent_system(llm)
print("🤖 双Agent系统初始化完成（支持 Hybrid 学生/教师 模式；若不可用则单模型）")


## 6. Notebook 中的使用示例（基础协作演示）
#
# 展示双 Agent 在 2 轮内对初稿进行学术优化的基本用法。
# 可修改 `sample_text`、`requirements` 或 `rounds` 观察行为变化。


In [ ]:
from pprint import pprint as _pprint_demo
sample_text = "这是一段关于多智能体协作进行学术写作优化的初稿，表述略显口语化，结构也不够清晰。"
requirements = ["学术表达提升", "逻辑结构优化"]
final_text_demo, log_demo = dual_agent_system.collaborate(sample_text, requirements, rounds=2)
print("\n📌 最终优化文本 (示例):\n")
print(final_text_demo)
print("\n📜 协作日志摘要（最后一轮）:\n")
_pprint_demo(log_demo[-1])


# 7. 高级功能：数据合成 / 评估 / 蒸馏 / 长文本优化 / HTML 报告
#
# 本节提供与脚本版本同步的高级接口：
# - dual_agent_system.synthesize_dataset(seeds, requirements, rounds)
# - dual_agent_system.evaluate([(text, reqs), ...], rounds)
# - dual_agent_system.prepare_distillation_pairs(jsonl_in, jsonl_out)
# - dual_agent_system.optimize_text_file(file_path, requirements, rounds, chunk_size, overlap)
# - dual_agent_system.generate_html_report(title, final_text, log, summary)
#
# 你可以根据需要在下方添加示例单元格进行调用。


# 8. 示例：评估与 HTML 报告生成（可自行取消注释运行）
# ```python
# samples = [
#     ("这是一个关于多智能体协作的初稿。", ["学术表达提升", "逻辑结构优化"]),
#     ("该方法在复杂场景下仍存在稳定性问题。", ["学术表达提升", "严谨性增强"])
# ]
# report = dual_agent_system.evaluate(samples, rounds=2)
# final_text, log = dual_agent_system.collaborate("多智能体优化系统可以提升写作质量。", ["学术表达提升"], rounds=2)
# html = dual_agent_system.generate_html_report("优化报告示例", final_text, log, report.get("summary"))
# open("report_example.html", "w", encoding="utf-8").write(html)
# html[:500]
# ```


# 9. 示例：长文本分段优化（可自行取消注释运行）
# ```python
# long_text = """你的很长的学术初稿..."""
# final_long, agg = dual_agent_system.optimize_text_file("path/to/long.txt", ["学术表达提升"], rounds=2, chunk_size=4000, overlap=200)
# len(final_long), agg['chunks']
# ```


# 10. 示例：数据合成与蒸馏对准备（可自行取消注释运行）
# ```python
# seeds = ["初稿段落 A", "初稿段落 B"]
# ds_path = dual_agent_system.synthesize_dataset(seeds, ["学术表达提升"], rounds=2)
# dual_agent_system.prepare_distillation_pairs(ds_path, "distill_pairs.jsonl")
# ```
